In [1]:
import spacy

# Lade das deutsche Sprachmodell von spaCy
nlp = spacy.load("de_core_news_sm")

def extract_keywords_from_query(query, custom_stopwords=None):
    """
    Extrahiert Keywords aus einer Query durch Tokenisierung und Entfernung von Stoppwörtern.
    """
    if custom_stopwords is None:
        custom_stopwords = set(nlp.Defaults.stop_words)
    
    # Tokenisiere die Query
    doc = nlp(query.lower())
    
    # Extrahiere Keywords: Wörter, die keine Stoppwörter, Zahlen oder Satzzeichen sind
    keywords = [token.text for token in doc if token.text not in custom_stopwords and not token.is_punct and not token.like_num]
    
    return keywords


In [2]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

# Lade das vortrainierte Modell
model_name = 'sentence-transformers/multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

# Initialisiere den Chroma-Client
chroma_client = chromadb.PersistentClient(
    path="chroma_db",
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)


# Überprüfe, ob die Collection existiert
def get_existing_collection(chroma_client, collection_name="document_embeddings_v2"):
    try:
        collection = chroma_client.get_collection(name=collection_name)
        print(f"Collection {collection_name} erfolgreich geladen.")
        return collection
    except ValueError:
        print(f"Collection {collection_name} existiert nicht.")
        return None

# Zugriff auf die bestehende Collection
collection = get_existing_collection(chroma_client, "document_embeddings_v2")

if collection is None:
    print("Die angegebene Collection existiert nicht. Bitte überprüfe den Collection-Namen.")
else:
    # Erweiterte Version der hybriden Suche mit normalisierter Distanzberechnung, Version 5
    def hybrid_search_v5(chroma_client, query, keywords, embedding_model, collection_name="document_embeddings_v2", top_k=10):
        # 1. Semantische Suche
        query_embedding = embedding_model.encode(query).tolist()
        collection = chroma_client.get_collection(name=collection_name)
        semantic_results = collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k
        )
        
        # 2. Distanznormalisierung
        distances = semantic_results['distances'][0]
        min_distance = min(distances)
        max_distance = max(distances)
        normalized_distances = [(d - min_distance) / (max_distance - min_distance) for d in distances]
        
        # 3. Klassische Textsuche basierend auf Keywords
        text_search_results = []
        for i in range(len(semantic_results['ids'][0])):
            metadata = semantic_results['metadatas'][0][i]
            keyword_count = sum(keyword.lower() in metadata['text'].lower() for keyword in keywords)
            if keyword_count > 0:
                text_search_results.append({
                    'id': semantic_results['ids'][0][i],
                    'text': metadata['text'],
                    'keywords': keywords,
                    'score': 1.0 + 0.5 * keyword_count  # Gewichtung für Keywords
                })
        
        # 4. Kombination der Ergebnisse
        combined_results = []
        seen_ids = set()
        for i in range(len(semantic_results['ids'][0])):
            distance = normalized_distances[i]
            combined_score = 1.0 - distance  # Gewichtung für semantische Suche
            
            if semantic_results['ids'][0][i] not in seen_ids:
                seen_ids.add(semantic_results['ids'][0][i])
                combined_results.append({
                    'id': semantic_results['ids'][0][i],
                    'text': semantic_results['metadatas'][0][i]['text'] if isinstance(semantic_results['metadatas'][0][i], dict) else "",
                    'keywords': keywords,
                    'score': combined_score
                })
        
        for result in text_search_results:
            if result['id'] not in seen_ids:
                combined_results.append(result)
        
        # 5. Sortieren der kombinierten Ergebnisse nach Score
        combined_results.sort(key=lambda x: x['score'], reverse=True)
        
        # 6. Ausgabe der Top-k Ergebnisse
        return combined_results[:top_k]

    # Beispielnutzung:
    query5 = "Was kommt in die gelbe Tonne?"
    # query5 = "Wie entsorge ich Altglas in Frankfurt?"

    keywords5 = extract_keywords_from_query(query5)

    # Durchführung der hybriden Suche mit Keywords
    hybrid_results_v5 = hybrid_search_v5(chroma_client, query5, keywords5, embedding_model)

    print(f"Query: {query5}")
    print(f"Extrahierte Keywords: {keywords5}")
    # Ausgabe der hybriden Suchergebnisse
    for i, result in enumerate(hybrid_results_v5):
        print(f"Ergebnis {i+1}:\n")
        print(f"ID: {result['id']}")
        print(f"Text: {result['text']}")
        print(f"Keywords: {result.get('keywords', [])}")
        print(f"Score: {result['score']}")
        print("\n" + "="*50 + "\n")

/Users/jan/Documents/GitHub/predictiv_chatbot/predictiv_chatbot/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/jan/Documents/GitHub/predictiv_chatbot/predictiv_chatbot/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Number of requested results 10 is greater than number of elements in index 8, updating n_results = 8


Collection document_embeddings_v2 erfolgreich geladen.
Query: Was kommt in die gelbe Tonne?
Extrahierte Keywords: ['gelbe', 'tonne']
Ergebnis 1:

ID: doc_7
Text: gelbe tonne gelben sack gehören gebrauchte restentleerte verpackungen nicht papier glas verpackungsbestandteile trennen ausspülen nicht menüschalen kunststoffdeckel fertiggerichten arzneimittelblister müsliriegelfolie joghurtbecher nudeltüten einkaufstüten reinigungsmittelflaschen gemüsebeutel kunststoff flüssigseife eisverpackungen fruchtpüree üllmaterial versand verpackungen shampooflaschen luftpolsterfolie spraydosen soßentüten konservendosen tierfutterdosen kronkorken zahnpastatuben kunststoffschalen lebensmittel nicht gelbe tonne gelben sack gehören verpackungen papier pappe glas abfälle verpackungen schuhe altkleider gummi strumpfhosen batterien holzwolle styroporreste behälterglas dämmplatten blechgeschirr tapetenreste disketten keramikprodukte windeln druckerpatronen kinderspielzeug zahnbürsten klarsichthüllen zigarett

In [ ]:
# Beispielnutzung:
query5 = "Wie entsorge ich Altglas in Frankfurt?"
keywords5 = extract_keywords_from_query(query5)

# Durchführung der hybriden Suche mit Keywords
hybrid_results_v5 = hybrid_search_v5(chroma_client, query5, keywords5, embedding_model)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from langchain_community.llms.ollama import Ollama

# Verwende das deutschsprachige Modell
model_name = "distilbert-base-german-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Initialisiere den Ollama-Client mit den angegebenen Daten
ollama_llm = Ollama(
    model="llama3.1:8b",
    base_url="http://192.168.180.131:11434"
)

# Initialisiere die Pipeline für Textklassifikation auf der GPU (falls verfügbar)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

def extract_relevant_information_transformers(text, threshold=0.5):
    """
    Verwendet ein Hugging Face Transformer-Modell, um relevante Informationen aus dem Text zu extrahieren.
    """
    sentences = text.split('. ')
    relevant_info = []

    for sentence in sentences:
        # Verwende das Modell, um die Relevanz jedes Satzes zu bestimmen
        result = classifier(sentence)
        # Füge relevante Sätze hinzu, basierend auf einem Schwellenwert
        if result[0]['score'] > threshold:
            relevant_info.append(sentence)
    
    return " ".join(relevant_info)

def build_filtered_context_transformers(hybrid_results, top_n=3):
    """
    Filtert und baut einen Kontextstring aus den besten Suchergebnissen auf,
    wobei ein Transformer-Modell zur Extraktion relevanter Informationen verwendet wird.
    """
    top_results = hybrid_results[:top_n]
    
    # Verwende das Transformer-Modell, um relevante Informationen zu extrahieren
    filtered_context = "\n\n".join([
        f"Result {i+1}:\n{extract_relevant_information_transformers(result['text'])}" 
        for i, result in enumerate(top_results)
    ])
    
    return filtered_context

def ask_llm_with_ollama(query, filtered_context, llm):
    """
    Sendet eine Query an das LLM (Ollama), zusammen mit dem gefilterten Kontext.
    
    Parameters:
    - query: Die ursprüngliche Frage.
    - filtered_context: Der gefilterte Kontext, der für die Beantwortung verwendet werden soll.
    - llm: Das LLM-Objekt, das die Anfrage bearbeitet.
    
    Returns:
    - Die Antwort des LLM.
    """
    prompt_text = f"""
        Frage: '{query}'

        Der folgende Text enthält wesentliche Informationen zur Mülltrennung in Deutschland. Beachte die spezifischen Kategorien und entscheide, welche Materialien in welche Tonne gehören.

        Kategorien:
        - Gelbe Tonne: Kunststoffverpackungen, Metallverpackungen, Verbundstoffe.
        - Papiertonne: Papier, Pappe.
        - Glascontainer: Glasflaschen, Gläser.
        - Restmüll: Dinge, die nicht recycelt werden können.
        - Biotonne: Organische Abfälle wie Lebensmittelreste und Gartenabfälle.

        Kontext:

        {filtered_context}

        Antwort:
        """
    
    response = llm(prompt_text)
    
    return response

# Beispielnutzung:
query5 = "Was kommt in die gelbe Tonne?"
keywords5 = extract_keywords_from_query(query5)

# Durchführung der hybriden Suche mit Keywords
hybrid_results_v5 = hybrid_search_v5(chroma_client, query5, keywords5, embedding_model)

# Baue den gefilterten Kontext mit Transformers
filtered_context_transformers = build_filtered_context_transformers(hybrid_results_v5, top_n=3)

# Sende die Query und den Transformer-gefilterten Kontext an Ollama
llm_response = ask_llm_with_ollama(query5, filtered_context_transformers, ollama_llm)

# Ausgabe der Antwort des LLM
print("Antwort des LLM:")
print(llm_response)

In [51]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from langchain_community.llms.ollama import Ollama

# Verwende das deutschsprachige Modell
model_name = "distilbert-base-german-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Initialisiere den Ollama-Client mit den angegebenen Daten
ollama_llm = Ollama(
    model="llama3.1:8b",
    base_url="http://192.168.180.131:11434"
)

# Initialisiere die Pipeline für Textklassifikation auf der GPU (falls verfügbar)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

def extract_relevant_information_transformers(text, threshold=0.5):
    """
    Verwendet ein Hugging Face Transformer-Modell, um relevante Informationen aus dem Text zu extrahieren.
    """
    sentences = text.split('. ')
    relevant_info = []

    for sentence in sentences:
        # Verwende das Modell, um die Relevanz jedes Satzes zu bestimmen
        result = classifier(sentence)
        # Füge relevante Sätze hinzu, basierend auf einem Schwellenwert
        if result[0]['score'] > threshold:
            relevant_info.append(sentence)
    
    return " ".join(relevant_info)

def build_filtered_context_transformers(hybrid_results, top_n=3):
    """
    Filtert und baut einen Kontextstring aus den besten Suchergebnissen auf,
    wobei ein Transformer-Modell zur Extraktion relevanter Informationen verwendet wird.
    """
    top_results = hybrid_results[:top_n]
    
    # Verwende das Transformer-Modell, um relevante Informationen zu extrahieren
    filtered_context = "\n\n".join([
        f"Result {i+1}:\n{extract_relevant_information_transformers(result['text'])}" 
        for i, result in enumerate(top_results)
    ])
    
    return filtered_context

def ask_llm_with_ollama(query, filtered_context, llm):
    """
    Sendet eine Query an das LLM (Ollama), zusammen mit dem gefilterten Kontext.
    
    Parameters:
    - query: Die ursprüngliche Frage.
    - filtered_context: Der gefilterte Kontext, der für die Beantwortung verwendet werden soll.
    - llm: Das LLM-Objekt, das die Anfrage bearbeitet.
    
    Returns:
    - Die Antwort des LLM.
    """
    prompt_text = f"""
    Frage: 


    Antwort:
    """
    
    response = llm(prompt_text)
    
    return response

# '{query}'

#    Bitte beantworte die folgende Frage direkt und spezifisch, basierend auf den bereitgestellten Informationen zur Mülltrennung. Stelle sicher, dass deine Antwort präzise, klar und ausschließlich auf den untenstehenden Kontext gestützt ist. Achte besonders auf Wörter wie 'außer', 'nicht' oder ähnliche Ausdrücke, die auf Ausnahmen oder Ausschlüsse hinweisen, um sicherzustellen, dass diese korrekt in die Antwort integriert werden. Vermeide Spekulationen oder Informationen, die nicht im Kontext enthalten sind. Gib die Antwort als Fließtext, ohne zu erwähnen, dass du aus einem Kontext zitierst oder auf spezifische Quellen verweist.

#    Kontext:

#    {filtered_context}
# Beispielnutzung:
#query5 = "Was kommt in die gelbe Tonne? Und was kommt in den Restmüll?"
query5 = "Wie entsorge ich Batterien?"

#query5 = "Wie werden Metalle wie Dosen und Kronkorken entsorgt?"
keywords5 = extract_keywords_from_query(query5)

# Durchführung der hybriden Suche mit Keywords
hybrid_results_v5 = hybrid_search_v5(chroma_client, query5, keywords5, embedding_model)

# Baue den gefilterten Kontext mit Transformers
filtered_context_transformers = build_filtered_context_transformers(hybrid_results_v5, top_n=3)

# Sende die Query und den Transformer-gefilterten Kontext an Ollama
llm_response = ask_llm_with_ollama(query5, filtered_context_transformers, ollama_llm)

# Ausgabe der Antwort des LLM
print(query5)
print("Antwort des Llama 3.1 8B:")
print(llm_response)
print("Ergebnisse aus RAG")
print(filtered_context_transformers)

/Users/jan/Documents/GitHub/predictiv_chatbot/predictiv_chatbot/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Number of requested results 10 is greater than number of elements in index 8, updating n_results = 8


Wie entsorge ich Batterien?
Antwort des Llama 3.1 8B:
Um Batterien ordnungsgemäß zu entsorgen, sollten sie aus Elektrogeräten entfernt und in spezielle Sammelboxen oder -stellen gegeben werden. Es besteht auch die Möglichkeit, sie in Wertstoffhöfen oder bei freigestellten Einrichtungen wie Tankstellen, Supermärkten und Drogeriemärkten abzugeben. Bei Fragen zur korrekten Mülltrennung kann an die kommunale Abfallberatung herangetreten werden. Altpapier hingegen sollte in der entsprechenden Container mit der blauen Tonne entsorgt werden, zusammen mit leeren Verpackungen, Zeitungen, Zeitschriften und Schulheften.
Ergebnisse aus RAG
Result 1:
batterien akkumulatoren gehören nicht brände auslösen umweltgefährdende stoffe enthalten mensch umwelt belasten ressourcen rohstoffe sammelboxen handel batterien verkauft supermärkte drogeriemärkte warenhäuser baumärkte tankstellen einpacktische kommunale sammelstellen wertstoffhöfe schadstoffmobile freiwillige sammelstellen unternehmen behörden hochsc

In [62]:
llm_response = ask_llm_with_ollama(query5, filtered_context_transformers, ollama_llm)
print(llm_response)

Batterien sollten in spezielle Sammelboxen oder sammelstellen, wie z.B. Tankstellen, Drogeriemärkte oder Baumärkte, gegeben werden und nicht mit Elektrogeräten entsorgt werden. Wenn du mehr über die korrekte Mülltrennung erfahren möchtest, kannst du dich an die kommunale Abfallberatung wenden.


In [53]:
# Test RAG
import pytest

# Beispiel-Implementierung des Testfalls
def test_rag_system():
    query5 = "Wie entsorge ich Batterien?"

    # Durchführung der Schritte in deiner RAG-Pipeline
    keywords5 = extract_keywords_from_query(query5)
    hybrid_results_v5 = hybrid_search_v5(chroma_client, query5, keywords5, embedding_model)
    filtered_context_transformers = build_filtered_context_transformers(hybrid_results_v5, top_n=3)
    llm_response = ask_llm_with_ollama(query5, filtered_context_transformers, ollama_llm)
    
    # Ausgabe der generierten Antwort
    print("LLM Response:", llm_response)
    
    # Überprüfen, ob die Antwort relevante Informationen enthält
    assert "Batterien" in llm_response, "Die Antwort sollte Informationen zur Entsorgung von Batterien enthalten."

# Ausführen des Tests in einem Jupyter-Notebook
test_rag_system()

Number of requested results 10 is greater than number of elements in index 8, updating n_results = 8


LLM Response: Batterien gehören nicht in die Blaue Tonne. Um sie ordnungsgemäß zu entsorgen, sollte man sie an Sammelstellen wie Kommunale Sammelstellen, Wertstoffhöfe, Schadstoffmobile oder Freiwillige Sammelstellen abgeben. Die genauen Informationen zur Batteriesammlung erhält man bei der kommunalen Abfallberatung.


In [60]:
# Importiere benötigte Module
import pytest
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer


# Unit-Tests
def test_extract_keywords():
    try:
        query = "Wie entsorge ich Glasflaschen?"
        keywords = extract_keywords_from_query(query)
        print("Extracted Keywords:", keywords)
        assert "glasflaschen" in keywords, "Schlüsselwort-Extraktion sollte 'glasflaschen' enthalten."
        print("Test 'test_extract_keywords' erfolgreich.")
    except AssertionError as e:
        print(f"Test 'test_extract_keywords' fehlgeschlagen: {e}")

def test_hybrid_search():
    try:
        query = "Wie entsorge ich Glasflaschen?"
        keywords = extract_keywords_from_query(query)
        results = hybrid_search_v5(chroma_client, query, keywords, embedding_model)
        assert len(results) > 0, "Hybrid-Suche sollte Ergebnisse liefern."
        print("Test 'test_hybrid_search' erfolgreich.")
    except AssertionError as e:
        print(f"Test 'test_hybrid_search' fehlgeschlagen: {e}")

def test_llm_response():
    try:
        query = "Wie entsorge ich Glasflaschen?"
        context = "Glasflaschen sollten in den Glascontainer entsorgt werden."
        response = ask_llm_with_ollama(query, context, ollama_llm)
        assert "Glasflaschen" in response, "LLM-Antwort sollte Informationen zu Glasflaschen enthalten."
        print("Test 'test_llm_response' erfolgreich.")
    except AssertionError as e:
        print(f"Test 'test_llm_response' fehlgeschlagen: {e}")

# Integrationstests
def test_full_pipeline_glasflaschen():
    try:
        query = "Wie entsorge ich Glasflaschen?"
        keywords = extract_keywords_from_query(query)
        hybrid_results = hybrid_search_v5(chroma_client, query, keywords, embedding_model)
        filtered_context = build_filtered_context_transformers(hybrid_results, top_n=3)
        response = ask_llm_with_ollama(query, filtered_context, ollama_llm)
        assert "Glasflaschen" in response, "Die Antwort sollte Informationen zur Entsorgung von Glasflaschen enthalten."
        print("Test 'test_full_pipeline_glasflaschen' erfolgreich.")
    except AssertionError as e:
        print(f"Test 'test_full_pipeline_glasflaschen' fehlgeschlagen: {e}")

def test_full_pipeline_batterien():
    try:
        query = "Wie entsorge ich Batterien?"
        keywords = extract_keywords_from_query(query)
        hybrid_results = hybrid_search_v5(chroma_client, query, keywords, embedding_model)
        filtered_context = build_filtered_context_transformers(hybrid_results, top_n=3)
        response = ask_llm_with_ollama(query, filtered_context, ollama_llm)
        assert "Batterien" in response, "Die Antwort sollte Informationen zur Entsorgung von Batterien enthalten."
        print("Test 'test_full_pipeline_batterien' erfolgreich.")
    except AssertionError as e:
        print(f"Test 'test_full_pipeline_batterien' fehlgeschlagen: {e}")

# Edge-Case-Tests
def test_empty_query():
    try:
        query = ""
        keywords = extract_keywords_from_query(query)
        assert keywords == [], "Schlüsselwort-Extraktion sollte eine leere Liste für leere Anfragen liefern."
        print("Test 'test_empty_query' erfolgreich.")
    except AssertionError as e:
        print(f"Test 'test_empty_query' fehlgeschlagen: {e}")

def test_unknown_query():
    try:
        query = "Wie entsorge ich Raumschiffmotoren?"
        keywords = extract_keywords_from_query(query)
        hybrid_results = hybrid_search_v5(chroma_client, query, keywords, embedding_model)
        filtered_context = build_filtered_context_transformers(hybrid_results, top_n=3)
        response = ask_llm_with_ollama(query, filtered_context, ollama_llm)
        assert "Raumschiff" not in response, "Die Antwort sollte keine falschen Informationen zu unbekannten Themen enthalten."
        print("Test 'test_unknown_query' erfolgreich.")
    except AssertionError as e:
        print(f"Test 'test_unknown_query' fehlgeschlagen: {e}")

def test_response_quality_glasflaschen():
    try:
        query = "Wie entsorge ich Glasflaschen?"
        expected_response = "Glasflaschen sollten in den Glascontainer entsorgt werden."
        
        keywords = extract_keywords_from_query(query)
        hybrid_results = hybrid_search_v5(chroma_client, query, keywords, embedding_model)
        filtered_context = build_filtered_context_transformers(hybrid_results, top_n=3)
        response = ask_llm_with_ollama(query, filtered_context, ollama_llm)
        
        scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        scores = scorer.score(expected_response, response)
        rouge_score = scores['rougeL'].fmeasure
        
        assert rouge_score > 0.7, "Die Qualität der Antwort sollte hoch sein."
        print("Test 'test_response_quality_glasflaschen' erfolgreich.")
    except AssertionError as e:
        print(f"Test 'test_response_quality_glasflaschen' fehlgeschlagen: {e}")
    except Exception as e:
        print(f"Ein Fehler ist im Test 'test_response_quality_glasflaschen' aufgetreten: {e}")


# Ausführen aller Tests in einem Jupyter-Notebook
def run_all_tests():
    test_extract_keywords()
    test_hybrid_search()
    test_llm_response()
    test_full_pipeline_glasflaschen()
    test_full_pipeline_batterien()
    test_empty_query()
    test_unknown_query()
    test_response_quality_glasflaschen()

run_all_tests()


Extracted Keywords: ['entsorge', 'glasflaschen']
Test 'test_extract_keywords' erfolgreich.


Number of requested results 10 is greater than number of elements in index 8, updating n_results = 8


Test 'test_hybrid_search' erfolgreich.
Test 'test_llm_response' erfolgreich.


Number of requested results 10 is greater than number of elements in index 8, updating n_results = 8


Test 'test_full_pipeline_glasflaschen' erfolgreich.


Number of requested results 10 is greater than number of elements in index 8, updating n_results = 8
Number of requested results 10 is greater than number of elements in index 8, updating n_results = 8


Test 'test_full_pipeline_batterien' erfolgreich.
Test 'test_empty_query' erfolgreich.


Number of requested results 10 is greater than number of elements in index 8, updating n_results = 8


Test 'test_unknown_query' fehlgeschlagen: Die Antwort sollte keine falschen Informationen zu unbekannten Themen enthalten.
Test 'test_response_quality_glasflaschen' fehlgeschlagen: Die Qualität der Antwort sollte hoch sein.
